In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/event_extraction/

/content/drive/MyDrive/event_extraction


In [ ]:
import os
import json
import torch
from torch.utils.data import DataLoader
from transformers import BartForConditionalGeneration, AutoTokenizer
from torch.optim import AdamW
from tqdm import tqdm
from src.event_argument_dataset import EventArgumentDataset
from src.eventtype_retriever import EventTypeRetriever
from src.utils.data_utils import build_labels, load_json_or_jsonl
from src.utils.device_util import getDeviceInfo
from src.utils.compute_metric import compute_metrics


In [ ]:

# ---------------------- Config ----------------------
DEVICE = getDeviceInfo()
BART_MODEL = "facebook/bart-base"
MAX_LENGTH = 256
OUTPUT_MAX_LENGTH = 64
BATCH_SIZE = 16
EPOCHS = 10
LR = 1e-5
TOP_K = 1
# CONTEXT_PATH = ""
CONTEXT_PATH = '/content/drive/MyDrive/event_extraction/'
CHECKPOINT_DIR = f"{CONTEXT_PATH}checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

TRAIN_JSON_PATH = f"{CONTEXT_PATH}processing_data/train.json"
VAL_JSON_PATH = f"{CONTEXT_PATH}processing_data/dev.json"
TEST_JSON_PATH = f"{CONTEXT_PATH}processing_data/test.json"
ONTOLOGY_PATH = f"{CONTEXT_PATH}ontoloy/event_role_WIKI_q.json"
LABEL_CACHE_PATH = f"{CONTEXT_PATH}processing_data/event_types.json"

event_types = build_labels(TRAIN_JSON_PATH, LABEL_CACHE_PATH)


Loaded labels from /content/drive/MyDrive/event_extraction/processing_data/event_types.json


In [ ]:
# ---------------------- Load tokenizer & model ----------------------
tokenizer = AutoTokenizer.from_pretrained(BART_MODEL)
special_tokens = ["<tgr>"]
tokenizer.add_tokens(special_tokens)

model = BartForConditionalGeneration.from_pretrained(BART_MODEL).to(DEVICE)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


BartScaledWordEmbedding(50266, 768, padding_idx=1)

In [ ]:
retriever = EventTypeRetriever(
    model_name=f"{CHECKPOINT_DIR}/retrieve_best_model",
    device=DEVICE,
    tokenizer=tokenizer,
    event_types=event_types,
    max_length=MAX_LENGTH
)

In [ ]:


# ---------------------- Load samples ----------------------
train_samples = load_json_or_jsonl(TRAIN_JSON_PATH)
val_samples = load_json_or_jsonl(VAL_JSON_PATH)
test_samples = load_json_or_jsonl(TEST_JSON_PATH)

# ---------------------- Dataset & DataLoader ----------------------
train_dataset = EventArgumentDataset(
    samples=train_samples,
    ontology_path=ONTOLOGY_PATH,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH,
    output_max_length=OUTPUT_MAX_LENGTH,
    topk_event_types=TOP_K,
    retriever=retriever
)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = EventArgumentDataset(
    samples=val_samples,
    ontology_path=ONTOLOGY_PATH,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH,
    output_max_length=OUTPUT_MAX_LENGTH,
    topk_event_types=TOP_K,
    retriever=retriever
)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

test_dataset = EventArgumentDataset(
    samples=test_samples,
    ontology_path=ONTOLOGY_PATH,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH,
    output_max_length=OUTPUT_MAX_LENGTH,
    topk_event_types=TOP_K,
    retriever=retriever
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:

# ---------------------- Hàm evaluate ----------------------
def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=OUTPUT_MAX_LENGTH)
            predictions = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]
            targets = [tokenizer.decode(t, skip_special_tokens=True) for t in labels]

            all_predictions.extend(predictions)
            all_targets.extend(targets)

    avg_loss = total_loss / len(loader)
    precision, recall, f1 = compute_metrics(all_predictions, all_targets)
    return avg_loss, all_predictions, all_targets, precision, recall, f1


In [ ]:

optimizer = AdamW(model.parameters(), lr=LR)
best_epoch = 4


In [ ]:
# ---------------------- Load best checkpoint để đánh giá test ----------------------
with open(os.path.join(CHECKPOINT_DIR, "best_checkpoint.txt"), "r") as f:
    best_epoch = int(f.read().strip())
best_ckpt_path = os.path.join(CHECKPOINT_DIR, f"bart_best_model_epoch{best_epoch}.pt")

model.load_state_dict(torch.load(best_ckpt_path))
model.to(DEVICE)
print(f"Loaded best model from epoch {best_epoch}")

Loaded best model from epoch 4


In [ ]:
# Evaluate
val_loss, val_preds, val_targets, p, r, f1 = evaluate(model, val_loader, DEVICE)
print(f"Val Loss: {val_loss:.4f} | Precision: {p:.4f} | Recall: {r:.4f} | F1: {f1:.4f}")

Evaluating: 100%|██████████| 97/97 [01:08<00:00,  1.42it/s]

Val Loss: 0.0156 | Precision: 0.3282 | Recall: 0.3457 | F1: 0.3263


In [ ]:
test_loss, test_preds, test_targets, p, r, f1 = evaluate(model, test_loader, DEVICE)
print(f"Test Loss: {test_loss:.4f} | Precision: {p:.4f} | Recall: {r:.4f} | F1: {f1:.4f}")

Evaluating: 100%|██████████| 108/108 [01:10<00:00,  1.52it/s]

Test Loss: 0.0165 | Precision: 0.3104 | Recall: 0.3236 | F1: 0.3060
